In [67]:
import pandas as pd
from scipy.io import arff
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import math
from sklearn import preprocessing
datatrain = arff.loadarff('trainProdSelection.arff')
dtrain = pd.DataFrame(datatrain[0])
str_df = dtrain.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
# dtrain['Type'].str.decode("utf-8")
for i in str_df:
    dtrain[i] = str_df[i]
# dtrain.head()

In [68]:
dtrain['Type'] = dtrain.Type.map({'student':1,'engineer':2,'librarian':3,'professor':4,'doctor':5})
dtrain['LifeStyle'] = dtrain.LifeStyle.map({'spend<<saving':1, 'spend<saving':2,'spend>saving':3, 'spend>>saving':4})
dtrain['label'] = dtrain.label.map({'C1':1,'C2':2,'C3':3,'C4':4,'C5':5})
dtrain.head()

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,1,3,6.0,40.0,13.62,3.2804,1
1,1,3,11.0,21.0,15.32,2.0232,1
2,1,3,7.0,64.0,16.55,3.1202,1
3,1,3,3.0,47.0,15.71,3.4022,1
4,1,3,15.0,10.0,16.96,2.2825,1


In [69]:
minV = dtrain.Vacation.min()
maxV = dtrain.Vacation.max()
dtrain.Vacation = dtrain.Vacation.apply(lambda x:(x-minV)/(maxV-minV))
mineC = dtrain.eCredit.min()
maxeC = dtrain.eCredit.max()
dtrain.eCredit = dtrain.eCredit.apply(lambda x:(x-mineC)/(maxeC-mineC))
minsalary = dtrain.salary.min()
maxsalary = dtrain.salary.max()
dtrain.salary = dtrain.salary.apply(lambda x:(x-minsalary)/(maxsalary-minsalary))
minp = dtrain.property.min()
maxp = dtrain.property.max()
dtrain.property = dtrain.property.apply(lambda x:(x-minp)/(maxp-minp))
dtrain.head() 

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,1,3,0.079365,0.107558,0.219960,0.183167,1
1,1,3,0.158730,0.052326,0.293102,0.112797,1
2,1,3,0.095238,0.177326,0.346023,0.174200,1
3,1,3,0.031746,0.127907,0.309882,0.189984,1
4,1,3,0.222222,0.020349,0.363663,0.127311,1


In [70]:
datatest = arff.loadarff("testProdSelection.arff")
dtest = pd.DataFrame(datatest[0])
str_df = dtest.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for i in str_df:
    dtest[i] = str_df[i]
# dtest.head()

In [71]:
dtest['Type'] = dtest.Type.map({'student':2,'engineer':3,'librarian':1,'professor':4,'doctor':5})
dtest['LifeStyle'] = dtest.LifeStyle.map({'spend<<saving':1, 'spend<saving':2,'spend>saving':3, 'spend>>saving':4})
dtest['label'] = dtest.label.map({'C1':1,'C2':2,'C3':3,'C4':4,'C5':5})
dtest.head()

,Type,LifeStyle,Vacation,eCredit,salary,property,label
0,2,2,12.0,19.0,14.7900,3.7697,1
1,2,4,29.0,10.0,16.1900,2.4839,1
2,2,1,28.0,60.0,15.4600,1.1885,1
3,3,3,15.0,41.0,21.2600,1.4379,1
4,1,2,2.0,9.0,19.7207,0.6913,1


In [72]:
minV = dtest.Vacation.min()
maxV = dtest.Vacation.max()
dtest.Vacation = dtest.Vacation.apply(lambda x:(x-minV)/(maxV-minV))
mineC = dtest.eCredit.min()
maxeC = dtest.eCredit.max()
dtest.eCredit = dtest.eCredit.apply(lambda x:(x-mineC)/(maxeC-mineC))
minsalary = dtest.salary.min()
maxsalary = dtest.salary.max()
dtest.salary = dtest.salary.apply(lambda x:(x-minsalary)/(maxsalary-minsalary))
minp = dtest.property.min()
maxp = dtest.property.max()
dtest.property = dtest.property.apply(lambda x:(x-minp)/(maxp-minp))
dtest.shape

(21, 7)

In [73]:
def euclideanDistance(data1,data2,length):
    distance = 0
    for i in range(length):
        distance += np.square(data1[i]-data2[i])
    return np.sqrt(distance)
def KNN(trainset,testset,k):
    distances = {}
    length = len(testset)-1
    for i in range(len(trainset)):
        dist = euclideanDistance(testset,trainset[i],length)
        dist=math.floor(dist)
        distances[i]=dist
    sortedd = sorted(distances.items(),key=operator.itemgetter(1))
    print(sortedd)
    neighbors = []
    for i in range(k):
        neighbors.append(sortedd[i])
    return(neighbors)
def getresponse(neis):
    classvotes = {}
    for i in range(len(neis)):
        response = neis[i][-1]
        if response in classvotes:
            classvotes[response]+=1
        else:
            classvotes[response] = 1
    sortedvotes = sorted(classvotes.items(),key = operator.itemgetter(1),reverse = True)
    print(sortedvotes)
    print("**********************")
    return sortedvotes[0][0]
def getaccuracy(testset,predict):
    correct = 0
    for x in range(len(testset)):
        if testset[x][-1] == predict[x]:
            correct += 1
    return (correct/float(len(testset))) * 100.0
        
k=7
predict = []
for i in range(len(dtest)):
    nei=KNN(dtrain.values,dtest.values[i],k)
    result = getresponse(nei)
    predict.append(result)
print(predict)
accuracy = getaccuracy(dtest.values,predict)
# print(nei)
print(accuracy)

[(145, 0), (147, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (113, 1), (114, 1), (115, 1), (116, 1), (117, 1), (119, 1), (129, 1), (131, 1), (133, 1), (134, 1), (135, 1), (136, 1), (137, 1), (139, 1), (141, 1), (143, 1), (144, 1), (146, 1), (149, 1), (168, 1), (169, 1), (170, 1), (171, 1), (172, 1), (173, 1), (174, 1), (175, 1), (176, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (26, 2), (27, 2), (28, 2), (29, 2), (50, 2), (51, 2), (52, 2), (53, 2), (54, 2), (55, 2), (56, 2), (57, 2), (58, 2), (59, 2), (60, 2), (61, 2), (112, 2), (118, 2), (120, 2), (121